# Security & Privacy (CC2009) - 2023/2024

## ASSIGNMENT #1: Performance Benchmarking of Cryptographic Mechanisms

### Due date: March 22, 23:59
### Grading: Assignment #1 is worth 2 points

In [1]:
import time
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def generate_random_text_file(size_bytes):
    return os.urandom(size_bytes)

def time_aes(file_sizes):
    backend = default_backend()
    key = os.urandom(32)  # 256-bit key
    iv = os.urandom(16)   # 128-bit IV
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=backend)
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(data) + encryptor.finalize()
        decryptor = cipher.decryptor()
        plaintext = decryptor.update(ciphertext) + decryptor.finalize()
        end_time = time.time()
        times.append(end_time - start_time)
    return times

def time_sha(file_sizes):
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        hash_obj = hashes.Hash(hashes.SHA256())
        hash_obj.update(data)
        hash_result = hash_obj.finalize()
        end_time = time.time()
        times.append(end_time - start_time)
    return times

def time_rsa(file_sizes):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    times = []
    for size in file_sizes:
        data = generate_random_text_file(size)
        start_time = time.time()
        ciphertext = public_key.encrypt(
            data,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        end_time = time.time()
        times.append(end_time - start_time)
    return times

# Sizes in bytes
file_sizes_aes = [8, 64, 512, 4096, 32768, 262144, 2097152]
file_sizes_sha = [8, 64, 512, 4096, 32768, 262144, 2097152]
file_sizes_rsa = [2, 4, 8, 16, 32, 64, 128]

print("AES Times:", time_aes(file_sizes_aes))
print("SHA Times:", time_sha(file_sizes_sha))
print("RSA Times:", time_rsa(file_sizes_rsa))


AES Times: [0.001834869384765625, 0.0001049041748046875, 4.291534423828125e-05, 4.1961669921875e-05, 0.00013327598571777344, 0.0007219314575195312, 0.006161212921142578]
SHA Times: [0.0002968311309814453, 6.9141387939453125e-06, 3.814697265625e-06, 1.3113021850585938e-05, 1.7881393432617188e-05, 0.00011205673217773438, 0.0008819103240966797]
RSA Times: [0.0025560855865478516, 0.0011568069458007812, 0.0011432170867919922, 0.0011410713195800781, 0.00109100341796875, 0.0010707378387451172, 0.001094818115234375]
